In [1]:
import sys 
sys.path.append('../backend')
sys.path.append('../')

In [2]:
# !pip install -r '../requirements.txt'

In [3]:
# !pip install qwen_vl_utils

In [4]:
# pip install torchvision

In [5]:
# from backend.models.vlm_qwen2b import *
# !pip install fastapi
from backend.main import *

/mnt/d/fun/vlm_app/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Using cache directory: /mnt/d/fun/vlm_app/backend/Weights


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  4.00s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Qwen2-VL model loaded successfully!


In [1]:
import requests

# Replace with the actual URL of your FastAPI application
BASE_URL = "http://localhost:8000"  # Or whatever port your app is running on

def health_check():
    """
    Performs a health check against the /health endpoint.
    """
    try:
        response = requests.get(f"{BASE_URL}/health")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error during health check: {e}")
        return None

if __name__ == "__main__":
    health_status = health_check()
    if health_status:
        print(f"Health Check: {health_status}")
    else:
        print("Health check failed.")

Health Check: {'status': 'ok'}


In [3]:
import requests
import io

def call_stream_process_image(image_path: str, text_prompt: str):
    """
    Calls the stream_process_image endpoint with an image and text prompt.

    Args:
        image_path: Path to the image file.
        text_prompt: The text prompt to send.

    Returns:
        The streaming response from the server.
    """
    try:
        with open(image_path, "rb") as image_file:
            files = {'files': (image_path, image_file, 'image/jpeg')} # Adjust image/jpeg to actual type if needed
            data = {'text_prompt': text_prompt}

            response = requests.post("http://localhost:8000/stream_process_image/", files=files, data=data, stream=True) # Replace with your actual URL

            if response.status_code == 200:
                for chunk in response.iter_content(chunk_size=512):  # Adjust chunk_size as needed
                    if chunk:
                        print(chunk.decode('utf-8', errors='ignore'), end='', flush=True)  # Print chunks
            else:
                print(f"Error: {response.status_code} - {response.text}")

    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
# Example Usage:
if __name__ == "__main__":
    image_file_path = "../data/bill_receipt.png"  # Replace with the actual path to your image
    user_prompt = "bill in the image"
    call_stream_process_image(image_file_path, user_prompt)

The image shows a cash receipt with the following details:

- **Shop Name**: Lorem Ipsum
- **Address**: Lorem Ipsum 3/18
- **Tel**: 0987 123 890 5678
- **Date**: MM/DD/YYYY
- **Manager**: Lorem Ipsum
- **Price**: 14.40
- **Tax**: 1.25
- **Total**: 15.65

There is also a barcode at the bottom of the receipt.